# DEA Notebook 

![alt text](../dea-gallery-crop.png "DEA")

### [DEA](https://dea.destine.eu/web) is a Content creation service and no-code platform for DestinE storytelling and data visualization

## Generate KMZ with TimeSpan from GeoTiff

This notebook helps generate Keyhole Markup Language Zipped (KMZ) files supported by the DEA service from sample GeoTIFF files.

The notebook will also include steps to add time information to the KMZ files and create time series from raster files.

>**_NOTE:_** You can replace the sample GeoTIFF files with your own GeoTIFF files.

# Install pre-requirements

In [2]:
!pip install GDAL
!pip install pillow
!pip install rasterio

## Define Your Inputs

The following 2 files generated from Sentinel-3 OLCI products related to the wildfire that occurred in Los Angeles in January 2025 will be used:

* LAWildfire_2025-01-09_S3_OLCI_Enhanced_Natural_Color.tiff
* LAWildfire_2025-01-10_S3_OLCI_Enhanced_Natural_Color.tiff

The sensing dates of the files are:

* 2025-01-09
* 2025-01-10


The GeoTIFF files are downloaded from the [Copernicus Data Space Ecosystem](https://browser.dataspace.copernicus.eu/)

In [19]:
# Input GeoTIFF files
tiff_file_inputs = ["LAWildfire_2025-01-09_S3_OLCI_Enhanced_Natural_Color.tiff", 
                    "LAWildfire_2025-01-10_S3_OLCI_Enhanced_Natural_Color.tiff"]      

# Time intervals to use
times = [("2025-01-09T00:00:00Z", "2025-01-09T23:59:59Z"), ("2025-01-10T00:00:00Z", "2025-01-10T23:59:59Z")]


## Convert Raster to PNG

In [20]:
import os
from PIL import Image

png_files = []

for tiff_file in tiff_file_inputs:
    print(f"Generating PNG from file {tiff_file}")
    png_path = os.path.splitext(tiff_file)[0] + '.png'
    
    with Image.open(tiff_file) as im:
        im = im.convert("RGBA")  # Ensure image has alpha channel
        datas = im.getdata()
        new_data = []
        
        for item in datas:
            # item is (R, G, B, A)
            if item[0] == 0 and item[1] == 0 and item[2] == 0:
                new_data.append((0, 0, 0, 0))  # Make black pixels fully transparent
            else:
                new_data.append(item)
        
        im.putdata(new_data)
        im.save(png_path, "PNG")
    print(f"PNG file {png_path} generated successfully")    
    png_files.append(png_path)



Generating PNG from file LAWildfire_2025-01-09_S3_OLCI_Enhanced_Natural_Color.tiff
PNG file LAWildfire_2025-01-09_S3_OLCI_Enhanced_Natural_Color.png generated successfully
Generating PNG from file LAWildfire_2025-01-10_S3_OLCI_Enhanced_Natural_Color.tiff
PNG file LAWildfire_2025-01-10_S3_OLCI_Enhanced_Natural_Color.png generated successfully


## Get Bounding Box from raster file

In [21]:
import rasterio

bboxes = []

for tiff_file in tiff_file_inputs:
    print(f"Getting Bounding Box from file {tiff_file}")
    with rasterio.open(tiff_file) as dataset:
        bounds = dataset.bounds
        # Bounds: left, bottom, right, top
        print(f"Bounding Box from file {tiff_file}: {bounds}")
        bboxes.append(bounds)




Getting Bounding Box from file LAWildfire_2025-01-09_S3_OLCI_Enhanced_Natural_Color.tiff
Bounding Box from file LAWildfire_2025-01-09_S3_OLCI_Enhanced_Natural_Color.tiff: BoundingBox(left=-120.981445, bottom=32.342756, right=-115.817871, top=35.586543)
Getting Bounding Box from file LAWildfire_2025-01-10_S3_OLCI_Enhanced_Natural_Color.tiff
Bounding Box from file LAWildfire_2025-01-10_S3_OLCI_Enhanced_Natural_Color.tiff: BoundingBox(left=-120.108032, bottom=32.912666, right=-116.965942, top=34.859111)


## Generate a KML with the PNG and the Time information

In [22]:
from xml.etree.ElementTree import Element, SubElement, tostring
from xml.dom.minidom import parseString

kml_files = []

for png_filename, (west, south, east, north), (begin, end) in zip(png_files, bboxes, times):
    base_name = os.path.splitext(os.path.basename(png_filename))[0]
    output_kml_path = os.path.join(os.path.dirname(png_filename), base_name + '.kml')
    kml_template = """<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://earth.google.com/kml/2.0">
<Document>
 <Folder>
    <name>Raster visibility</name>
    <TimeSpan><begin>{begin}</begin><end>{end}</end></TimeSpan>
    <Folder>
      <name>Raster</name>
      <GroundOverlay>
          <name>Raster data</name>
          <LatLonBox>
            <north>{north}</north>
            <south>{south}</south>
            <west>{west}</west>
            <east>{east}</east>
          </LatLonBox>
          <Icon>
            <href>{href}</href>
          </Icon>
      </GroundOverlay>
    </Folder>
  </Folder>
</Document>
</kml>"""
    filled_kml = kml_template.format(
        begin=begin,
        end=end,
        north=north,
        south=south,
        west=west,
        east=east,
        href=png_filename
    )
    with open(output_kml_path, 'w') as f:
        f.write(filled_kml)
    print(f"KML file {output_kml_path} generated successfully")
    kml_files.append(output_kml_path)

KML file LAWildfire_2025-01-09_S3_OLCI_Enhanced_Natural_Color.kml generated successfully
KML file LAWildfire_2025-01-10_S3_OLCI_Enhanced_Natural_Color.kml generated successfully


## Generate the KMZ file

In [23]:
import zipfile

for png_path, kml_path in zip(png_files, kml_files):
    base_name = os.path.splitext(os.path.basename(png_path))[0]
    kmz_path = os.path.join(os.path.dirname(png_path), base_name + '.kmz')
    with zipfile.ZipFile(kmz_path, 'w', zipfile.ZIP_DEFLATED) as kmz:
        kmz.write(png_path, os.path.basename(png_path))
        kmz.write(kml_path, os.path.basename(kml_path))
    print(f"KMZ created at: {kmz_path}")

KMZ created at: LAWildfire_2025-01-09_S3_OLCI_Enhanced_Natural_Color.kmz
KMZ created at: LAWildfire_2025-01-10_S3_OLCI_Enhanced_Natural_Color.kmz


# Upload the KMZ files on DEA and Set Autoplay

### You can now upload the generated KMZ files on the [DEA Story Editor](https://dea.destine.eu/stories/editor).

![alt text](dea_upload_assets.png "Upload Assets on DEA")

### The asset can be included in your stories for creating animations.

<div>
    <center>
        <img src="add_asset_to_slide.png" width="400" alt="Add asset to Slide"/>
    </center>
</div>

### Set the slide date range according to the time interval of your newly created time series.

![alt text](set_slide_date_range.png "Set the slide date range")

### Enable Autoplay and set a suitable Speed to automatically see your time series' animation in the DEA Story Viewer.

<div>
    <center>
        <img src="dea_settings.png" width="400" alt="Enable Autoplay and set Speed"/>
    </center>
</div>

<center style="font-size: 40pt;"><b>Hope you love it!</b></center>

<div>
    <center>
        <img src="tiff2kmz.gif" width="600"/>
    </center>
</div>